In [1]:
import concurrent.futures
import datetime
import os

import pandas as pd
import torch
from moviepy.editor import VideoFileClip
from pytube import YouTube

In [2]:
TIME_DURATION_IN_SEC = 10
CSV_FILE = './vggsound.csv'

In [3]:
# Rename the columns
new_column_names = {
    '---g-f_I2yQ': 'youtube_video_id',
    '1': 'start_seconds',
    'people marching': 'label',
    'test': 'split',
}

In [5]:
# Rename the columns
df = pd.read_csv(CSV_FILE)
df.rename(columns=new_column_names, inplace=True)

In [6]:
df.head()

,youtube_video_id,start_seconds,label,split
0,--0PQM4-hqg,30,waterfall burbling,train
1,--56QUhyDQM,185,playing tennis,train
2,--5OkAjCI7g,40,people belly laughing,train
3,--8puiAGLhs,30,car engine starting,train
4,--96EN9NUQM,242,alarm clock ringing,train


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "false" if device == "cpu" else "true"

In [14]:
def extract_audio_and_video_within_interval(youtube_video_id, input_video_path, start_time, end_time):
    # Load the video clip
    video_clip = VideoFileClip(input_video_path)

    # Define the subclip with the specified start and end times
    subclip = video_clip.subclip(start_time, end_time)

    # Extract audio from the subclip
    audio_clip = subclip.audio

    # Write the audio to a WAV file

    audio_file_path = f"./data/audio/audio_{youtube_video_id}.wav"
    video_file_path = f"./data/video/video_{youtube_video_id}.mp4"

    audio_clip.write_audiofile(audio_file_path, codec='pcm_s16le', fps=audio_clip.fps)
    subclip.write_videofile(video_file_path, codec='libx264')

    # Close the clips
    video_clip.close()
    audio_clip.close()

In [15]:
def download_files(youtube_video_id, start_seconds):
    print(f"Starting to download file: {youtube_video_id}")

    # Construct the YouTube video URL
    video_url = f"https://www.youtube.com/watch?v={youtube_video_id}"

    # Initialize a YouTube object with the URL
    yt = YouTube(video_url)

    downloaded_file = f"{youtube_video_id}.mp4"

    try:
        # Get a stream that includes both video and audio
        stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

        # Download the video
        stream.download(output_path=".", filename=downloaded_file)

        print(f"Downloaded video: {youtube_video_id}")

        # Specify the start and end times for clipping (in seconds)
        input_file_path = downloaded_file
        extract_audio_and_video_within_interval(youtube_video_id, input_file_path, start_seconds,
                                                start_seconds + TIME_DURATION_IN_SEC)

        print(f"Saved audio and video for: {youtube_video_id}")

    except Exception as e:
        print(f"Error downloading video with YouTube ID {youtube_video_id}: {e}")

    if os.path.exists(f"./{downloaded_file}"):
        os.remove(f"./{downloaded_file}")

In [16]:
def extract_audio_video(limit=2):
    start_time = datetime.datetime.now()
    # Create a ThreadPoolExecutor with 3 worker threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        # Submit each video downloading task for execution asynchronously
        futures = [executor.submit(download_files, row['youtube_video_id'], row['start_seconds']) for index, row in
                   df.iterrows() if index <= limit]

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

    end_time = datetime.datetime.now()

    total_time = (end_time - start_time) / 60

    print(f"Time taken for download: {total_time} minutes")

In [17]:
count = 5

In [18]:
extract_audio_video(count)

Starting to download file: --0PQM4-hqg
Starting to download file: --56QUhyDQM
Starting to download file: --5OkAjCI7g
Error downloading video with YouTube ID --5OkAjCI7g: --5OkAjCI7g is a private video
Starting to download file: --8puiAGLhs
Downloaded video: --0PQM4-hqg
Downloaded video: --8puiAGLhs
MoviePy - Writing audio in ./data/audio/audio_--0PQM4-hqg.wav


MoviePy - Done.
Moviepy - Building video ./data/video/video_--0PQM4-hqg.mp4.
MoviePy - Writing audio in video_--0PQM4-hqgTEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 1/221 [00:00<00:01, 124.83it/s, now=None]

MoviePy - Writing audio in ./data/audio/audio_--8puiAGLhs.wav



chunk:   1%|          | 2/221 [00:00<00:12, 16.99it/s, now=None]

...

MoviePy - Done.
Moviepy - Writing video ./data/video/video_--96EN9NUQM.mp4




t:  98%|█████████▊| 234/240 [00:06<00:00, 26.55it/s, now=None][A
                                                              [A
t:  34%|███▍      | 103/300 [00:03<00:10, 19.25it/s, now=None]
                                                           
t:  35%|███▍      | 104/300 [00:03<00:10, 19.25it/s, now=None]
                                                           
t:  35%|███▌      | 106/300 [00:03<00:12, 16.03it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./data/video/video_--9O4XZOge4.mp4
Saved audio and video for: --9O4XZOge4



t:  99%|█████████▊| 296/300 [00:08<00:00, 32.44it/s, now=None]
                                                              

Moviepy - Done !
Moviepy - video ready ./data/video/video_--96EN9NUQM.mp4
Saved audio and video for: --96EN9NUQM
Time taken for download: 0:00:00.651886 minutes
